In [2]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00


In [18]:
!pip install pyxlsb

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import pymysql

from datetime import datetime, timedelta

def excel_a_fecha(valor_excel):
    # Convertir a fecha
    fecha_excel = datetime(1899, 12, 30) + timedelta(days=valor_excel)

    # Devolver la fecha
    return fecha_excel

In [327]:
ruta = "/content/drive/MyDrive/lap/RiesgoCartera/"

In [2]:
cnx = pymysql.connect(host='159.89.90.197', port=3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')

**Se carga toda la cartera**

In [5]:
sql = """
select * from
cartera_distribuidor
where fecha = '2024-05-06'
"""
cursor = cnx.cursor()
cursor.execute(sql)
data_cartera = cursor.fetchall()
data_cartera = pd.DataFrame(data_cartera, columns=[i[0] for i in cursor.description])

In [6]:
data_cartera['id_distribuidor'].nunique()

19502

## Se cruza la base anterior con la actual para definirle el riesgo

In [53]:
data_enviada_actual = pd.read_excel("./data/cartera_DISTRIBUIDORES_CIERRE04ABR24_002.xlsx", 
                                    sheet_name='REPORTE CARTERA', header=1)

In [54]:
data_enviada = pd.read_excel("./data/240115 Riesgo ponderado de la cartera actual.xlsb")

In [55]:
def get_risk(id_):
    query = data_enviada[data_enviada['ID DIST.'] == id_]
    if len(query) == 0:
        return np.nan
    
    return query['NIVEL DE RIESGO'].values[0]

In [58]:
data_enviada_actual['NIVEL DE RIESGO'] = data_enviada_actual['ID DIST.'].apply(get_risk)

data_enviada = data_enviada_actual.copy()

In [61]:
np.sum(data_enviada['NIVEL DE RIESGO'].isna()) / len(data_enviada)

0.6471105896157231

**Se carga la BD dada**

In [63]:
data_enviada.rename(columns = {"NIVEL DE RIESGO":"riesgo"}, inplace = True)
data_enviada_completa = data_enviada.query(f"not riesgo.isna()").copy()
data_enviada = data_enviada.query(f"riesgo.isna()")

In [64]:
data_enviada.shape

(2206, 105)

**Se carga la última base de riesgo**

In [66]:
data_riesgo_ant = pd.read_csv("./data/riesgo_cartera_2024_01_18.csv")

In [67]:
data_riesgo_ant['id_distribuidor'].nunique()

17253

In [68]:
faltantes = data_enviada['ID DIST.']#np.setdiff1d(data_enviada['ID DIST.'].unique(), data_riesgo_ant['id_distribuidor'].unique())

In [69]:
data_cartera['capital'] = data_cartera.apply(lambda row: row['total_pagado_capital'] + row['saldo_pendiente_capital'],
                                             axis = 1)

In [70]:
np.setdiff1d(faltantes, data_cartera.id_distribuidor)

array([], dtype=int64)

In [71]:
data_cosecha = data_cartera.groupby('id_distribuidor').agg({'capital':'sum',
                                                            'saldo_pendiente_capital':'sum',
                                                            'fecha_aut_linea':'max'}).reset_index()


In [72]:
data_cosecha = data_cosecha.loc[[k in np.array(faltantes) for k in np.array(data_cosecha['id_distribuidor'])]]

In [73]:
data_cosecha['perdida_cosecha'] = 1 - (data_cosecha['capital'] - data_cosecha['saldo_pendiente_capital']) /  data_cosecha['capital']

In [74]:
data_cosecha.fecha_aut_linea = pd.to_datetime(data_cosecha.fecha_aut_linea)

In [75]:
from datetime import datetime, timedelta

def han_pasado_menos_de_cuatro_meses(fecha_dada):
    # Obtener la fecha actual
    fecha_actual = datetime.now()

    # Calcular la diferencia en meses
    diferencia_meses = (fecha_actual.year - fecha_dada.year) * 12 + fecha_actual.month - fecha_dada.month

    # Verificar si han pasado menos de 4 meses
    return diferencia_meses < 4

data_cosecha['menos_cuatro_meses'] = data_cosecha.fecha_aut_linea.apply(han_pasado_menos_de_cuatro_meses)


In [76]:
data_cosecha.perdida_cosecha.mean()

0.6759474245800962

In [77]:
data_cosecha = data_cosecha[['id_distribuidor', 'perdida_cosecha', 'menos_cuatro_meses']]

In [78]:
def setcut(cosecha_):
    #p = p*100
    cosecha_ *= 100

    grupo = 1
    cuts = np.array(sorted([100, 81.09, 72.06, 63.19, 54.27, 45.11, 36.10, 27.47, 18.17, 9.05, 0.00]))
    cosechas = np.array(sorted([0, 1.54, 3.4, 4.97, 7.38, 10.69, 11.87, 12.35, 12.68, 20.33, 22.69], reverse = True))

    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))

    enter_ = False
    for cosecha in cosechas:
        if cosecha <= cosecha_:

            enter_ = True
            break

    if enter_:
        grupo = np.where(cosechas == cosecha)[0][0]

    return grupo, cuts[grupo], cosechas[grupo]

In [79]:
def setcut_prob(p, grupo_ = True):
    p = p*100
    cuts = np.array(sorted([100, 81.09, 72.06, 63.19, 54.27, 45.11, 36.10, 27.47, 18.17, 9.05, 0.00], reverse = True))
    cosechas = np.array(sorted([1.54, 3.4, 4.97, 7.38, 10.69, 11.87, 12.35, 12.68, 20.33, 22.69]))

    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))

    for cut in cuts:
        if cut < p:
            break
    grupo = np.where(cuts == cut)[0][0]
    if grupo_:
        return grupo
    else:
        return cosechas[grupo - 1]

In [80]:
_, data_cosecha['probs'], data_cosecha['cosecha_canasta'] = zip(*data_cosecha.perdida_cosecha.apply(setcut))

In [81]:
def move_prob(val, menos_cuatro):
    if menos_cuatro:
      return np.random.randint(300, 850)/1000
    val /= 100
    if val == 0:
        val += np.random.randint(0, 200)/1000
    elif val == 1:
        val -= np.random.randint(0, 200)/1000
    else:
        val += np.random.randint(-200, 200)/1000
    return val

In [82]:
data_cosecha.probs = data_cosecha.apply(lambda row: move_prob(row['probs'], row['menos_cuatro_meses']), axis = 1)

In [83]:
data_cosecha

,id_distribuidor,perdida_cosecha,menos_cuatro_meses,probs,cosecha_canasta
7010,113640,1.0,False,0.064,22.69
8581,122691,1.0,False,0.112,22.69
8607,122811,1.0,False,0.167,22.69
9415,126589,1.0,False,0.174,22.69
15287,161191,NaN,True,0.836,20.33
...,...,...,...,...,...
19485,186813,1.0,True,0.563,22.69
19487,186834,1.0,True,0.755,22.69
19488,186840,NaN,True,0.726,20.33
19489,186842,1.0,True,0.461,22.69


In [84]:
data_cosecha['grupo'] = data_cosecha.probs.apply(setcut_prob, grupo_ = True)

In [85]:
data_cosecha['cosecha_canasta'] = data_cosecha.probs.apply(setcut_prob, grupo_ = False)

In [86]:
data_cosecha.drop("perdida_cosecha", axis = 1, inplace=True)
data_cosecha.drop("menos_cuatro_meses", axis = 1, inplace=True)

In [87]:
data_cosecha

,id_distribuidor,probs,cosecha_canasta,grupo
7010,113640,0.064,22.69,10
8581,122691,0.112,20.33,9
8607,122811,0.167,20.33,9
9415,126589,0.174,20.33,9
15287,161191,0.836,1.54,1
...,...,...,...,...
19485,186813,0.563,7.38,4
19487,186834,0.755,3.40,2
19488,186840,0.726,3.40,2
19489,186842,0.461,10.69,5


In [434]:
#data_riesgo_ant.probs = data_riesgo_ant.probs / 100

In [88]:
data_riesgo_ant['cosecha_canasta'] = data_riesgo_ant.probs.apply(setcut_prob, grupo_ = False)

In [89]:
data_riesgo_ant.rename(columns={'ID DIST.': 'id_distribuidor'}, inplace = True)
data_riesgo_ant = data_riesgo_ant[['id_distribuidor', 'probs', 'grupo', 'cosecha_canasta']];

In [90]:
data_cosecha = data_cosecha[['id_distribuidor', 'probs', 'grupo', 'cosecha_canasta']]

In [91]:
data_final = pd.concat([data_riesgo_ant, data_cosecha])

In [92]:
data_final

,id_distribuidor,probs,grupo,cosecha_canasta
0,1,0.8019,2,3.40
1,19,0.0885,10,22.69
2,29,0.0795,10,22.69
3,38,0.1777,9,20.33
4,51,0.9870,1,1.54
...,...,...,...,...
19485,186813,0.5630,4,7.38
19487,186834,0.7550,2,3.40
19488,186840,0.7260,2,3.40
19489,186842,0.4610,5,10.69


In [93]:
# Guardando para la próxima
data_final.to_csv("./data/riesgo_cartera_2024_05_08.csv")

In [97]:
data_final

,id_distribuidor,probs,grupo,cosecha_canasta
0,1,0.8019,2,3.40
1,19,0.0885,10,22.69
2,29,0.0795,10,22.69
3,38,0.1777,9,20.33
4,51,0.9870,1,1.54
...,...,...,...,...
19485,186813,0.5630,4,7.38
19487,186834,0.7550,2,3.40
19488,186840,0.7260,2,3.40
19489,186842,0.4610,5,10.69


**Uniendo lso grupos al archivo**

In [94]:
data_cosecha[['id_distribuidor', 'grupo']]

,id_distribuidor,grupo
7010,113640,10
8581,122691,9
8607,122811,9
9415,126589,9
15287,161191,1
...,...,...
19485,186813,4
19487,186834,2
19488,186840,2
19489,186842,5


In [95]:
for _, row in data_cosecha[['id_distribuidor', 'grupo']].iterrows():
  data_enviada.loc[data_enviada['ID DIST.'] == row['id_distribuidor'], "riesgo"] = row['grupo']

In [96]:
#Concatenar con la original
data_enviada

,SUCURSAL,ID COORDINADOR,COORDINADOR,ID ASESOR,ASESOR,ID DIST.,DISTRIBUIDOR,F. AUT. LINEA,F. ULT. INCREMENTO,NO. INCREMENTOS,...,DIAS ATRASO (CIERRECOMERCIAL),DIAS ATRASO (ABRIL),DIAS ATRASO (MARZO),BUCKET ABRIL,BUCKET MARZO,REGION,AJUSTE CONTABLE MARZO 24,x,x.1,riesgo
354,COATZACOALCOS,35481,TEMPORAL-ISABEL MONSERRAT RAZO MENDEZ,14019,TEMPORAL-ISABEL MONSERRAT RAZO MENDEZ,113640,FELICITA BUSTAMANTE DE LA CRUZ,2023-02-15,NaT,0,...,179,181,NaN,NaN,NaN,REGIÓN F,NaN,NaN,NaN,10.0
384,COATZACOALCOS,35481,TEMPORAL-ISABEL MONSERRAT RAZO MENDEZ,14019,TEMPORAL-ISABEL MONSERRAT RAZO MENDEZ,122691,AURORA EPITACIO ROMAN,2022-11-09,NaT,0,...,179,181,NaN,NaN,NaN,REGIÓN F,NaN,NaN,NaN,9.0
386,COATZACOALCOS,35481,TEMPORAL-ISABEL MONSERRAT RAZO MENDEZ,33951,TEMPORAL-JULIO CESAR GONZALEZ ESPINOZA,122811,FABIOLA MENDEZ GUZMAN,2022-07-27,NaT,0,...,179,181,NaN,NaN,NaN,REGIÓN F,NaN,NaN,NaN,9.0
406,COATZACOALCOS,35481,TEMPORAL-ISABEL MONSERRAT RAZO MENDEZ,14019,TEMPORAL-ISABEL MONSERRAT RAZO MENDEZ,126589,FELICIANA VASQUEZ RAMIREZ,2022-08-31,NaT,0,...,179,181,NaN,NaN,NaN,REGIÓN F,NaN,NaN,NaN,9.0
955,TONALA,31406,TEMPORAL-MIGUEL AGUSTIN CHAVEZ HUITRON,38113,YENNI GUADALUPE CANO PEREZ,161191,JAZMIN NATALIA NARVAEZ GONZALEZ,2024-02-26,NaT,0,...,0,0,0.0,0 días,0 días,REGIÓN O,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,CHIMALHUACAN 2,36217,MARIA ESTHER RAMIREZ CASTAÑEDA,35609,BRENDA BERENICE CASTRO HERNANDEZ,186753,BERISIMA ESCAMILLA GOMEZ,2024-04-30,NaT,0,...,0,0,NaN,0 días,NaN,REGIÓN O,NaN,NaN,NaN,4.0
3405,CHIMALHUACAN 2,36217,MARIA ESTHER RAMIREZ CASTAÑEDA,35609,BRENDA BERENICE CASTRO HERNANDEZ,186803,LLUVIA ANGELICA SAN JUAN GUZMAN,2024-04-30,NaT,0,...,0,0,NaN,0 días,NaN,REGIÓN O,NaN,NaN,NaN,7.0
3406,CHIMALHUACAN 2,36217,MARIA ESTHER RAMIREZ CASTAÑEDA,35609,BRENDA BERENICE CASTRO HERNANDEZ,186813,Topografos,2024-04-30,NaT,0,...,0,0,NaN,0 días,NaN,REGIÓN O,NaN,NaN,NaN,4.0
3407,CHIMALHUACAN 2,36217,MARIA ESTHER RAMIREZ CASTAÑEDA,35609,BRENDA BERENICE CASTRO HERNANDEZ,186840,ANA GABRIELA MATIAS BLAS,2024-04-30,NaT,0,...,0,0,NaN,0 días,NaN,REGIÓN O,NaN,NaN,NaN,2.0


In [445]:
data_final_enviada = pd.concat([data_enviada_completa, data_enviada])

In [451]:
data_final_enviada.to_csv(ruta + "/data/240118 Riesgo ponderado actualizado de la cartera actual.csv", index = False)